In [1]:
import pandas as pd
import re
import ast
from sklearn.impute import SimpleImputer

In [ ]:
job_df = pd.read_csv('job_cleaned.csv')
job_df['programming language'] = job_df['programming language'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)
job_df = job_df[job_df['programming language'].str.len() > 0].reset_index(drop=True)
job_df.info()

In [ ]:
d = {}

def note_numlanguage(languages):
    for lang_list in languages:
        for lang in lang_list:  
            if lang in d:
                d[lang] += 1  
            else:
                d[lang] = 1  

note_numlanguage(job_df['programming language'])

language_df = pd.DataFrame(columns=['programming language', 'job amount'])
print(len(d))
for key, value in d.items():
    new_row ={'programming language':key, 'job amount':value}
    language_df = language_df._append(new_row, ignore_index=True)
    print(f"{key}: {value}")

In [ ]:
language_list = language_df['programming language'].to_list()
language_salary_dict={}
language_location_dict={}

for lang in language_list:
    language_salary_dict[lang] = 0
    language_location_dict[lang] = []

print(language_salary_dict)

In [ ]:
for _, row in job_df.iterrows():
    salary = row['salary']
    for lang in row['programming language']:
        language_salary_dict[lang] += salary

print(language_salary_dict)

In [ ]:
amount_list = language_df['job amount'].to_list()

i=0
for key,val in language_salary_dict.items():
    language_salary_dict[key] = val/amount_list[i]
    i+=1

print(language_salary_dict)


In [ ]:
language_salary_df = pd.DataFrame(list(language_salary_dict.items()), columns=['programming language', 'average salary'])
language_df = pd.merge(language_df, language_salary_df, on='programming language')

language_df.head()

In [ ]:
for _, row in job_df.iterrows():
    location = row['location']
    for lang in row['programming language']:
        cleaned_text = re.sub(r'\b\w+(?:[\s-]\w+)*,\s*', '', location)
        if(cleaned_text!='Indonesia'): 
            if('Yogyakarta' in cleaned_text): cleaned_text='Yogyakarta'
            language_location_dict[lang].append(cleaned_text)
        else:
            match = re.search(r'^\s*(\w+(?:[\s-]\w+)*)\s*,', location)
            if match: language_location_dict[lang].append(match.group(1))
            else: continue


print(language_location_dict)

In [ ]:
for key, lis in language_location_dict.items():
    lis2 = list(set(lis))
    language_location_dict[key] = lis2

print(language_location_dict)

In [ ]:
language_location_df = pd.DataFrame(list(language_location_dict.items()), columns=['programming language', 'location'])
language_df = pd.merge(language_df, language_location_df, on='programming language')

language_df.head()

In [13]:
language_df.to_csv('language_list.csv', index=False)

In [ ]:
language_df.describe()